In [ ]:
from ase.optimize.fire import FIRE
from ase.io import write
from multiple_minimum_monte_carlo.conformer import Conformer
from multiple_minimum_monte_carlo.calculation import ASEOptimization
from multiple_minimum_monte_carlo.conformer_ensemble import ConformerEnsemble

To use this code, you need an ASE calculator. For this example, we'll use the AIMNet2 calculator. This isn't installed with this package so you'll need to install it separately to run this example! (pip install git+https://github.com/isayevlab/aimnetcentral.git)

In [ ]:
from aimnet.calculators import AIMNet2ASE

Run a multiple minimum monte carlo search from a SMILES string (will work for any ase calculator and optimizer combination, here we use xtb and FIRE)

In [ ]:
smiles = "CC(=O)Oc1ccccc1C(=O)O"
conformer = Conformer(smiles=smiles)
optimizer = ASEOptimization(calc=AIMNet2ASE(), optimizer=FIRE, fmax=0.03, verbose=False)
conformer_ensemble = ConformerEnsemble(conformer=conformer, calc=optimizer, verbose=True, num_iterations=10)
conformer_ensemble.run_monte_carlo()
conformer.atoms.set_positions(conformer_ensemble.final_ensemble[0])
write("lowest_energy_conformer.xyz", conformer.atoms, format="xyz")
print(f"Lowest energy conformer energy: {conformer_ensemble.final_energies[0]} kcal/mol")

Run a multiple minimum monte carlos search from an input XYZ file. If you don't have a SMILES string, it will attempt to generate a mol from the XYZ file (this may fail for more complex molecules)

In [ ]:
input_xyz = "example.xyz"
conformer = Conformer(input_xyz=input_xyz, charge=0)
optimizer = ASEOptimization(calc=AIMNet2ASE(), optimizer=FIRE)
conformer_ensemble = ConformerEnsemble(conformer=conformer, calc=optimizer, verbose=True, num_iterations=30, initial_optimization=False)
conformer_ensemble.run_monte_carlo()
conformer.atoms.set_positions(conformer_ensemble.final_ensemble[0])
write("lowest_energy_conformer_from_xyz.xyz", conformer.atoms, format="xyz")

Parallel calculations are implemented specifically with the "fork" start method from multiprocessing to get around pickling errors. If this is incompatible with your workflow, parallel calculations won't work!

In [ ]:
import multiprocessing as mp

print(mp.get_start_method())  # Should print 'fork'
smiles = "CC(=O)Oc1ccccc1C(=O)O"
conformer = Conformer(smiles=smiles)
optimizer = ASEOptimization(calc=AIMNet2ASE(), optimizer=FIRE, fmax=0.03, verbose=False)
conformer_ensemble = ConformerEnsemble(conformer=conformer, calc=optimizer, verbose=True, num_iterations=10, parallel=True, num_cpus=4)
conformer_ensemble.run_monte_carlo()
conformer.atoms.set_positions(conformer_ensemble.final_ensemble[0])
write("lowest_energy_conformer.xyz", conformer.atoms, format="xyz")
print(f"Lowest energy conformer energy: {conformer_ensemble.final_energies[0]} kcal/mol")

To run batch calculations, we will need to use a batch calculator which will involve installing a couple more packages that aren't installed with this package. We will use the TorchSim calculator here (to use, pip install torch-sim-atomistic). For this example, we will use the uma-s-1 (to use, pip install fairchem-core and request access through HuggingFace).

In [ ]:
from multiple_minimum_monte_carlo.batch_calculation import TorchSimCalculation
from torch_sim.models.fairchem import FairChemModel
from torch_sim.optimizers import Optimizer


smiles = "CC(=O)Oc1ccccc1C(=O)O"
conformer = Conformer(smiles=smiles)
model = FairChemModel(model=None, model_name="uma-s-1",task_name="omol", cpu=True)
calc = TorchSimCalculation(model=model, optimizer=Optimizer.fire, max_cycles=500)
conformer_ensemble = ConformerEnsemble(conformer=conformer, calc=calc, verbose=True, num_iterations=10, batch_size=5)
conformer_ensemble.run_monte_carlo()
conformer.atoms.set_positions(conformer_ensemble.final_ensemble[0])
write("lowest_energy_conformer_batch.xyz", conformer.atoms, format="xyz")
print(f"Lowest energy conformer energy: {conformer_ensemble.final_energies[0]} kcal/mol")